<p align="left">
  <img src="https://raw.githubusercontent.com/python35/IINTS-SDK/main/img/iints_logo.png" width="160">
</p>

# Quickstart: Run a Full Simulation
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/00_Quickstart.ipynb)

**Goal:** run an end-to-end simulation and get a results CSV, audit trail, baseline comparison, and PDF report.

**You will learn:**
- Load a preset scenario
- Apply a demo-safe patient profile
- Run the full pipeline with `iints.run_full`
- Locate the output files

**Expected output:** a `results/quickstart` folder with CSV + audit + baseline + PDF.


In [1]:
from __future__ import annotations
from pathlib import Path
from typing import Optional
import os
import sys
import subprocess


def _find_repo_root() -> Optional[Path]:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


Repo root: /home/runner/work/IINTS-SDK/IINTS-SDK


## Optional: AI Research Track (Predictor, Not a Controller)

IINTS-AF includes a research pipeline that trains a glucose prediction model and feeds the Safety Supervisor with a 30-120 minute forecast. The predictor never doses insulin; it only provides a forecast signal.

Install research extras:
```bash
pip install iints-sdk-python35[research]
```

Train a starter predictor (synthetic bootstrap):
```bash
python research/synthesize_dataset.py --runs 25 --output data/synthetic.parquet
python research/train_predictor.py --data data/synthetic.parquet --config research/configs/predictor.yaml --out models
```

Integrate the predictor in a run:
```python
from iints.research import load_predictor_service
predictor = load_predictor_service("models/predictor.pt")
outputs = iints.run_simulation(..., predictor=predictor)
```

## Step 1: Load a preset and algorithm


In [2]:
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


## Step 2: Apply a demo‑safe patient profile


In [3]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


## Step 3: Run full pipeline (audit + PDF + baseline)


In [4]:
outputs = iints.run_full(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=360,
    time_step=preset["time_step_minutes"],
    seed=42,
    output_dir="results/quickstart",
    enable_profiling=True,
)

outputs.keys()


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


dict_keys(['results', 'safety_report', 'run_id', 'output_dir', 'baseline_comparison', 'config_path', 'run_metadata_path', 'results_csv', 'audit', 'baseline_files', 'report_pdf', 'profiling_path', 'run_manifest_path'])

## Step 4: Inspect outputs


In [5]:
outputs["results"].head()


,time_minutes,glucose_actual_mgdl,glucose_to_algo_mgdl,glucose_trend_mgdl_min,predicted_glucose_30min,predicted_glucose_heuristic_30min,predicted_glucose_ai_30min,delivered_insulin_units,algo_recommended_insulin_units,sensor_status,...,safety_level,safety_actions,safety_reason,safety_triggered,supervisor_latency_ms,human_intervention,human_intervention_note,algorithm_why_log,algorithm_latency_ms,step_latency_ms
0,0,150.000000,150.000000,0.000000,150.000000,150.000000,None,0.033333,0.033333,ok,...,safe,,APPROVED,False,0.014823,False,,[],0.002153,0.071902
1,5,142.495556,142.495556,-1.500889,97.272222,97.272222,None,0.033333,0.033333,ok,...,safe,,APPROVED,False,0.009837,False,,[],0.001425,0.060793
2,10,135.357444,135.357444,-1.427622,92.138778,92.138778,None,0.033333,0.033333,ok,...,safe,,APPROVED,False,0.007779,False,,[],0.000971,0.035490
3,15,128.562906,128.562906,-1.358908,87.215672,87.215672,None,0.033333,0.033333,ok,...,safe,,APPROVED,False,0.006999,False,,[],0.000963,0.031565
4,20,122.090316,122.090316,-1.294518,82.488111,82.488111,None,0.033333,0.033333,ok,...,safe,,APPROVED,False,0.006538,False,,[],0.000839,0.028655


In [6]:
print("Report PDF:", outputs.get("report_pdf"))
print("Audit summary:", outputs.get("audit", {}).get("summary"))


Report PDF: /home/runner/work/IINTS-SDK/IINTS-SDK/results/quickstart/clinical_report.pdf
Audit summary: /home/runner/work/IINTS-SDK/IINTS-SDK/results/quickstart/audit/audit_summary.json


### Recap
You now have a results CSV, an audit trail, a baseline comparison, and a PDF report in `results/quickstart/`.
